## Modeling Baseline

In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import random
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

## Seed 고정 Gpu 설정

In [5]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Load Tokenizer, Model
Hugging Face Hub에 존재하는 Pretrained Tokenizer와 Model 및 Model Config를 불러옵니다.

In [11]:
MODEL_NAME = 'tunib/electra-ko-base'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = AutoConfig.from_pretrained(MODEL_NAME)
config.num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config = config)

Some weights of the model checkpoint at tunib/electra-ko-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at tunib/electra-ko-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [12]:
config1

ElectraConfig {
  "_name_or_path": "tunib/electra-ko-base",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

### Tokenizing
Train Data를 Train과 Validation Dataset으로 나누고 각각 데이터를 Tokenizer를 통해 Tokenizing을 합니다.

In [15]:
train_dataset = pd.read_csv("/home/egg2018037024/Interlink_Project/Data/train_data.csv", encoding='cp949', index_col=0)
eval_dataset = pd.read_csv("/home/egg2018037024/Interlink_Project/Data/eval_data.csv", encoding='cp949', index_col=0)

tokenized_train = tokenizer(
    list(train_dataset['Sentence']),
    return_tensors="pt",  # pytorch.Tensor로 리턴
    max_length=128, # Max_Length = 75
    padding=True,  # batch 중 가장 긴 시퀀스를 기준으로 pad 채움.
    truncation=True,  # max_length 넘어가면 버림
    add_special_tokens=True
)

tokenized_eval = tokenizer(
    list(eval_dataset['Sentence']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print(tokenized_train['input_ids'][1])
print(tokenizer.decode(tokenized_train['input_ids'][1]))

tensor([    2,    34,  4818, 28889, 16477,  6433,  6067,    34,  1537,  6563,
         6125, 13758,  6044,  4528, 23039, 30092, 19479,    34,    34,    14,
           14,    62,    56,  6033,  6064,    63,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
[CLS]? 카지노 눈치백단? 똘똘한 담당자 차실장 상시대기?? + + [ URL ] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

## Dataset 클래스 정의

In [16]:
class Elec_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label):  # 전처리된 데이터 셋이 들어옴
        self.dataset = dataset
        self.label = label

    def __getitem__(self, idx):
        # gradient 계산에 영향을 주지 않게 clone().detach() 실행
        
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):  # 샘플 수
        return len(self.label)

In [18]:
train_label = train_dataset['Label']
eval_label = eval_dataset['Label']

In [19]:
train_dataset = Elec_Dataset(tokenized_train, train_label)
eval_dataset = Elec_Dataset(tokenized_eval, eval_label)

print(train_dataset.__len__())
print(eval_dataset.__len__())
print(train_dataset.__getitem__(19997))
print(tokenizer.decode(train_dataset.__getitem__(19997)['input_ids']))

48030
5337
{'input_ids': tensor([    2,  2180,  4818,  3153,  1864,  4245,  5432,  4976,   925,  4774,
         5436, 14518,    62,    56,  6033,  6064,    63,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
def compute_metrics(pred):
    """ validation을 위한 metrics function """

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = pred.predictions

    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds) 

    return {
      'accuracy': acc,
    }

## Trainer 정의

In [22]:
# model

training_ars = TrainingArguments(
    output_dir='./result',
    num_train_epochs=3,
    max_steps=10000,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    save_total_limit=5,
    save_steps=1000,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy='steps',
    eval_steps = 1000,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=model,
    args=training_ars,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [23]:
trainer.train()
model.save_pretrained('./result/best_model')

/home/egg2018037024/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 48030
  Num Epochs = 54
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 10000
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
1000,0.002100,0.003388,0.999251
2000,0.000400,0.003634,0.999438
3000,0.000300,0.004962,0.999063
4000,0.000000,0.001707,0.999063
5000,0.000100,0.005218,0.999063
6000,0.000100,0.004290,0.999251
7000,0.000000,0.003290,0.999251
8000,0.000000,0.003846,0.999251
9000,0.000000,0.003789,0.999251
10000,0.000000,0.003464,0.999251


***** Running Evaluation *****
  Num examples = 5337
  Batch size = 256
Saving model checkpoint to ./result/checkpoint-1000
Configuration saved in ./result/checkpoint-1000/config.json
Model weights saved in ./result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./result/checkpoint-1000/special_tokens_map.json
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 5337
  Batch size = 256
Saving model checkpoint to ./result/checkpoint-2000
Configuration saved in ./result/checkpoint-2000/config.json
Model weights saved in ./result/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./result/checkpoint-2000/t

## 추론

In [45]:
test_sen = ["아직 개인으로 하고 계신가요? 손실은 감당이 되세요? 월 중순 수익 볼 수 있게 일정을 맞췄습니다^^ <우리투자 빅 이벤트> 현재 시가총액 위인 ○○하이닉스는 현대전자로 설립,  ○○하이닉스로 상호를 변경했습니다. 무료거부"]
#test_sen = ["아침식사 해야지? 집에올거야?"]

In [46]:
test = pd.DataFrame(test_sen, columns=['Sentence'])

In [48]:
test['Label'] = 1

test

,Sentence,Label
0,아직 개인으로 하고 계신가요? 손실은 감당이 되세요? 월 중순 수익 볼 수 있게 일...,1


In [49]:
MODEL_NAME = './result/best_model'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(tokenizer.vocab_size)
model.to(device)

loading configuration file ./result/best_model/config.json
Model config ElectraConfig {
  "_name_or_path": "./result/best_model",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file ./result/best_model/pytorch_model.bin
All model c

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [50]:
test_label = test['Label']

In [51]:
tokenized_test = tokenizer(
    list(test['Sentence']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = Elec_Dataset(tokenized_test,test_label)

print(test_dataset.__len__())
#print(test_dataset.__getitem__(1665))
#print(tokenizer.decode(test_dataset.__getitem__(6)['input_ids']))

1


In [52]:
dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
output_pred = []
output_prob = []

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'].to(device),
            attention_mask=data['attention_mask'].to(device),
            token_type_ids=data['token_type_ids'].to(device)
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 45.47it/s]

[1]
